In [0]:
# !mkdir ./detections/
# !wget -O ./detections/detections.zip http://jacarini.dinf.usherbrooke.ca/static/pedestrian%20detection//pedestrian%20detection%20dataset.zip
# !unzip ./detections/detections.zip -d ./detections/
# !rm ./detections/detections.zip

In [0]:
files = {}
aa = lambda x, y: list(range(x, y))
files["backdoor"] = aa(57, 2000)
files["pedestrians"] = aa(0, 1090)
files["PETS2006"] = aa(0, 1200)
files["peopleInShade"] = aa(0, 1190)

gt = {}
gt["backdoor"] = [[] for i in range(0, 2000)]
gt["pedestrians"] = [[] for i in range(0, 1090)]
gt["PETS2006"] = [[] for i in range(0, 1200)]
gt["peopleInShade"] = [[] for i in range(0, 1190)]

start = {"backdoor": [0], 
         "pedestrians": [0],
         "PETS2006": [0],
         "peopleInShade": [0]}

# files = {}
# aa = lambda x, y: list(range(x, y))

# # files["skating"] = aa(0, 3900)
# # files["sofa"] = aa(0, 2750)
# files["busStation"] = aa(0, 1250)

# gt = {}
# # gt["skating"] = [[] for i in range(0, 3901)]
# # gt["sofa"] = [[] for i in range(0, 2751)]
# gt["busStation"] = [[] for i in range(0, 1251)]

# start = {"skating": [0], 
#          "sofa": [0],
#          "busStation": [0]}

In [0]:
for key in gt:
    with open("/content/detections/pedestrian detection dataset/" + key + "/gt.txt") as f:
        detections = f.readlines()
        for detection in detections:
            items = detection.split()
            if len(items) == 0:
                continue
            frameID = int(items[0])
            if frameID < start[key][0]:
                continue
            x0 = int(float(items[2]))
            y0 = int(float(items[3]))
            w = int(float(items[4]))
            if frameID == 445 and key == "PETS2006":
                h = 82
            elif frameID == 938 and key == "sofa":
                h = 126
            else:
                h = int(float(items[5]))
            gt[key][frameID-start[key][0]].append([x0, y0, x0+w, y0+h])
            

In [0]:
from PIL import Image
import torch
import torchvision
import torchvision.transforms as T
import numpy as np

from torchvision.models.detection.transform import GeneralizedRCNNTransform
from torchvision.models.detection.generalized_rcnn import GeneralizedRCNN

from collections import OrderedDict
from torch import nn
import warnings
from torch.jit.annotations import Tuple, List, Dict, Optional
from torch import Tensor
from copy import deepcopy
from torchvision.utils import save_image


In [0]:
# NetA = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False) 
# NetA.transform = GeneralizedRCNNTransform(min_size=400,
#                                      max_size=800,
#                                      image_mean = [0.485, 0.456, 0.406],
#                                      image_std = [0.229, 0.224, 0.225])
# NetA.roi_heads.box_predictor.cls_score = nn.Linear(in_features=1024, out_features=2, bias=True)
# NetA.roi_heads.box_predictor.bbox_pred = nn.Linear(in_features=1024, out_features=8, bias=True)

# NetA.load_state_dict(torch.load("/content/drive/My Drive/889PR/bestA.pth"))

In [0]:
class NETB(GeneralizedRCNN):
    def __init__(self, backbone, rpn, roi_heads, transform):
        super().__init__(backbone=backbone, 
                         rpn=rpn, 
                         roi_heads=roi_heads, 
                         transform=transform)
    
    def forward(self, xs, ys, targets=None):
        if self.training and targets is None:
            raise ValueError("In training mode, targets should be passed")
        original_image_sizes = torch.jit.annotate(List[Tuple[int, int]], [])
        for img in xs:
            val = img.shape[-2:]
            assert len(val) == 2
            original_image_sizes.append((val[0], val[1]))

        images = []
        for i in range(len(xs)):
            images.append(torch.cat((xs[i], ys[i]), dim=0))
        images, targets = self.transform(images, targets)
        
        features = self.backbone(images.tensors)
        if isinstance(features, torch.Tensor):
            features = OrderedDict([('0', features)])
        
        proposals, proposal_losses = self.rpn(images, features, targets)
        detections, detector_losses = self.roi_heads(features, proposals, images.image_sizes, targets)
        detections = self.transform.postprocess(detections, images.image_sizes, original_image_sizes)

        losses = {}
        losses.update(detector_losses)
        losses.update(proposal_losses)

        if torch.jit.is_scripting():
            if not self._has_warned:
                warnings.warn("RCNN always returns a (Losses, Detections) tuple in scripting")
                self._has_warned = True
            return (losses, detections)
        else:
            return self.eager_outputs(losses, detections)

NetB = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)   
backbone = NetB.backbone
transform = GeneralizedRCNNTransform(min_size=400,
                                     max_size=800,
                                     image_mean = [0.485, 0.456, 0.406, 0.485, 0.456, 0.406],
                                     image_std = [0.229, 0.224, 0.225, 0.229, 0.224, 0.225])
rpn = NetB.rpn
roi_heads = NetB.roi_heads
backbone.body.conv1 = nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
roi_heads.box_predictor.cls_score = nn.Linear(in_features=1024, out_features=2, bias=True)
roi_heads.box_predictor.bbox_pred = nn.Linear(in_features=1024, out_features=8, bias=True)

NetB = NETB(backbone=backbone, rpn=rpn, roi_heads=roi_heads, transform=transform)
NetB.load_state_dict(torch.load("/content/drive/My Drive/889PR/bestBrandom.pth"))

<All keys matched successfully>

In [0]:
# from collections import OrderedDict as OD
# class NETC(GeneralizedRCNN):
#     def __init__(self, backbone, rpn, roi_heads, transform, STEPS):
#         super().__init__(backbone=backbone, 
#                          rpn=rpn, 
#                          roi_heads=roi_heads, 
#                          transform=transform)
#         self.MergeNet = nn.Sequential(nn.Conv2d(2*STEPS, 2*STEPS, 1, bias=False),
#                                       nn.BatchNorm2d(2*STEPS, affine=False),
#                                       nn.ReLU(inplace=True),
#                                       nn.Conv2d(2*STEPS, STEPS, 1, bias=False),
#                                       nn.BatchNorm2d(STEPS, affine=False),
#                                       nn.ReLU(inplace=True))
#         self.STEPS = STEPS
#     def forward(self, xs, ys, targets=None):
#         STEPS = self.STEPS
#         if self.training and targets is None:
#             raise ValueError("In training mode, targets should be passed")
#         original_image_sizes = torch.jit.annotate(List[Tuple[int, int]], [])
#         for img in xs:
#             val = img.shape[-2:]
#             assert len(val) == 2
#             original_image_sizes.append((val[0], val[1]))

#         xs, targets = self.transform(xs, targets)
#         ys, _ = self.transform(ys, None)
        
#         features1 = self.backbone(xs.tensors)
#         features2 = self.backbone(ys.tensors)
#         features3 = OD()
#         for key in features1:
#             features3[key] = torch.zeros(features1[key].size()).cuda()

#         for key in features3:
#             for i in range(features3[key].size(0)):
#                 for j in range(0, features3[key].size(1), STEPS):
#                     inp = torch.zeros(1, 2 * STEPS, features1[key].size(2), features1[key].size(3))
#                     inp[0, 0 : 2 * STEPS : 2, :, :] = features1[key][i, j:j+STEPS, :, :].unsqueeze(0)
#                     inp[0, 1 : 2 * STEPS : 2, :, :] = features2[key][i, j:j+STEPS, :, :].unsqueeze(0)
#                     features3[key][i,j:j+STEPS,:,:] = self.MergeNet(inp.cuda()).unsqueeze(0)       
        
#         proposals, proposal_losses = self.rpn(xs, features3, targets)
#         detections, detector_losses = self.roi_heads(features3, proposals, xs.image_sizes, targets)
#         detections = self.transform.postprocess(detections, xs.image_sizes, original_image_sizes)

#         losses = {}
#         losses.update(detector_losses)
#         losses.update(proposal_losses)

#         if torch.jit.is_scripting():
#             if not self._has_warned:
#                 warnings.warn("RCNN always returns a (Losses, Detections) tuple in scripting")
#                 self._has_warned = True
#             return (losses, detections)
#         else:
#             return self.eager_outputs(losses, detections)
# NetC = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)   
# backbone = NetC.backbone
# transform = GeneralizedRCNNTransform(min_size=400,
#                                      max_size=800,
#                                      image_mean = [0.485, 0.456, 0.406],
#                                      image_std = [0.229, 0.224, 0.225])
# rpn = NetC.rpn
# roi_heads = NetC.roi_heads
# roi_heads.box_predictor.cls_score = nn.Linear(in_features=1024, out_features=2, bias=True)
# roi_heads.box_predictor.bbox_pred = nn.Linear(in_features=1024, out_features=8, bias=True)
# steps = 64
# NetC = NETC(backbone=backbone, rpn=rpn, roi_heads=roi_heads, transform=transform, STEPS=steps)
# pth = "/content/drive/My Drive/889PR/bestCrandom" + str(2*steps) +  "to" + str(steps) + ".pth"
# NetC.load_state_dict(torch.load(pth))

In [0]:
# from collections import OrderedDict as OD
# class NETD(GeneralizedRCNN):
#     def __init__(self, backbone, rpn, roi_heads, transform):
#         super().__init__(backbone=backbone, 
#                          rpn=rpn, 
#                          roi_heads=roi_heads, 
#                          transform=transform)
    
#     def forward(self, xs, ys, targets=None):
#         if self.training and targets is None:
#             raise ValueError("In training mode, targets should be passed")
#         original_image_sizes = torch.jit.annotate(List[Tuple[int, int]], [])
#         for img in xs:
#             val = img.shape[-2:]
#             assert len(val) == 2
#             original_image_sizes.append((val[0], val[1]))

#         xs, targets = self.transform(xs, targets)
#         ys, _ = self.transform(ys, None)
        
#         features1 = self.backbone(xs.tensors)
#         features2 = self.backbone(ys.tensors)
#         features3 = OD()
#         for key in features1:
#             features3[key] = torch.max(features1[key], features2[key])
        
#         proposals, proposal_losses = self.rpn(xs, features3, targets)
#         detections, detector_losses = self.roi_heads(features3, proposals, xs.image_sizes, targets)
#         detections = self.transform.postprocess(detections, xs.image_sizes, original_image_sizes)

#         losses = {}
#         losses.update(detector_losses)
#         losses.update(proposal_losses)

#         if torch.jit.is_scripting():
#             if not self._has_warned:
#                 warnings.warn("RCNN always returns a (Losses, Detections) tuple in scripting")
#                 self._has_warned = True
#             return (losses, detections)
#         else:
#             return self.eager_outputs(losses, detections)

# NetD = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)   
# backbone = NetD.backbone
# transform = GeneralizedRCNNTransform(min_size=300,
#                                      max_size=800,
#                                      image_mean = [0.485, 0.456, 0.406],
#                                      image_std = [0.229, 0.224, 0.225])
# rpn = NetD.rpn
# roi_heads = NetD.roi_heads
# roi_heads.box_predictor.cls_score = nn.Linear(in_features=1024, out_features=2, bias=True)
# roi_heads.box_predictor.bbox_pred = nn.Linear(in_features=1024, out_features=8, bias=True)

# NetD = NETD(backbone=backbone, rpn=rpn, roi_heads=roi_heads, transform=transform)
# NetD.load_state_dict(torch.load("/content/drive/My Drive/889PR/bestDmax.pth"))

In [0]:
def IOU(boxA, boxB):
    l = max(boxA[0], boxB[0])
    r = min(boxA[2], boxB[2])
    t = max(boxA[1], boxB[1])
    b = max(boxA[3], boxB[3])

    intersection = max(r-l, 0) * max(b-t, 0)
    union = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1]) + (boxB[2] - boxB[0]) * (boxB[3] - boxB[1]) - intersection
    return intersection / union

def SCORE(gts, dets):
    if len(gts) == 0:
        return 0, len(dets), 0
    if len(dets) == 0:
        return 0, 0, len(gts)
    scores = np.zeros((len(dets), len(gts)))
    mask = np.ones_like(scores)
    for i, gt in enumerate(gts):
        for j, det in enumerate(dets):
            scores[j, i] = IOU(gt, det)
    TP, FP, FN = 0, 0, 0
    t = []
    while np.max(mask*scores) > 0:
        loc = np.argmax(mask * scores)
        val = np.max(mask * scores)
        mask[loc//mask.shape[1], :] = 0
        mask[:, loc%mask.shape[1]] = 0
        # print(mask)
        if val > 0.5:
            TP += 1
    FN = len(gts) - TP
    FP = len(dets) - TP
    return TP, FP, FN


In [0]:
model = NetB
batch_size = 5
def imread(path, num):
    im = Image.open(path + str(num).zfill(6) + ".jpg")
    transform = T.Compose([T.ToTensor()])
    im = transform(im)
    return im.cuda()

model.cuda()
model.eval()

metrics = {}
metrics["PETS2006"] = {"TP": 0, "FN": [0, []], "FP": [0, []]}
metrics["backdoor"] = {"TP": 0, "FN": [0, []], "FP": [0, []]}
metrics["pedestrians"] = {"TP": 0, "FN": [0, []], "FP": [0, []]}
metrics["peopleInShade"] = {"TP": 0, "FN": [0, []], "FP": [0, []]}
# metrics["skating"] = {"TP": 0, "FN": [0, []], "FP": [0, []]}
# metrics["sofa"] = {"TP": 0, "FN": [0, []], "FP": [0, []]}
# metrics["busStation"] = {"TP": 0, "FN": [0, []], "FP": [0, []]}

for key in files:
    bg = imread("/content/detections/pedestrian detection dataset/" + key +"/input/in", 1)
    for i in range(0, len(files[key]), batch_size):
        if i % 20 == 0:
            save_image(bg, key + str(i) + '.png')
        print("\r %s: [%d / %d]" % (key, i, len(files[key])), end="")
        ids = files[key][i:i+batch_size]
        sample = [imread("/content/detections/pedestrian detection dataset/" + key +"/input/in", num+1) for num in ids]
        detections = model(sample, [bg]*10)
        # detections = model(sample)
        results = []
        for detection in detections:
            result = []
            scores = detection["scores"].detach().cpu()
            boxes = detection["boxes"].detach().cpu()
            for j, score in enumerate(scores):
                if score > 0.5:
                    result.append(boxes[j].numpy())
            results.append(result)
        mask = torch.zeros(bg.size()).cuda()
        
        if len(results[-1]) > 0:
            result = results[-1]
            for re in result:
                x0 = re[0]
                y0 = re[1]
                x1 = re[2]
                y1 = re[3]
                mask[:, int(x0):int(x1), int(y0):int(y1)] = 1
        bg = 1-mask * bg + (1-mask) * sample[-1]
        gts = gt[key][i:i+batch_size]
        for j, gtt in enumerate(gts):
            tp, fp, fn = SCORE(gtt, results[j])
            metrics[key]["TP"] +=  tp
            metrics[key]["FP"][0] +=  fp
            metrics[key]["FN"][0] +=  fn
            if fp > 0:
                metrics[key]["FP"][1].append(ids[j])
            if fn > 0:
                metrics[key]["FN"][1].append(ids[j])
    print("")

 backdoor: [0 / 1943]

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


 backdoor: [165 / 1943]

KeyboardInterrupt: ignored

In [0]:
bg.size()

torch.Size([3, 240, 320])

In [0]:
filename = 'NetC8b.txt'
# filename = 'NetDb.txt'
for key in metrics:
    with open(filename, 'a') as f:
        print(key, file=f)
    with open(filename, 'a') as f:
        print("TP:", metrics[key]["TP"], file=f)
    with open(filename, 'a') as f:
        print("FP:", metrics[key]["FP"][0], file=f)
    with open(filename, 'a') as f:
        print("FN:", metrics[key]["FN"][0], file=f)
    with open(filename, 'a') as f:
        print("FP:", metrics[key]["FP"][1], file=f)
    with open(filename, 'a') as f:
        print("FN:", metrics[key]["FN"][1], file=f)
    with open(filename, 'a') as f:   
        print("--" * 20, file=f)

In [0]:
results